# Retail Sales Analysis

This notebook processes retail sales data.

**Project Overview:**

- **Data Sources**: Branches, Articles (Products), and Sales Transactions
- **Technology Stack**: Python, Pandas, SQLite
- **Analysis Period**: Last 6 months of sales data

**Note:** This is a basic implementation that needs refactoring for production use.

# Learning Objectives

By refactoring this notebook, you will:

- **Python Skills**: Practice data manipulation with Pandas, database operations with SQLite, and writing clean, maintainable Python code
- **SQL Proficiency**: Refresh SQL skills through writing joins, aggregations, and complex queries for business analytics
- **Software Engineering Best Practices**:
  - Apply PEP 8 coding standards for consistent, readable code
  - Implement comprehensive error handling and logging
  - Write modular, reusable code with proper separation of concerns
  - Create unit and integration tests using pytest
- **Code Quality**: Transform exploratory notebook code into production-ready modules
- **Performance Optimization**: Identify and resolve performance bottlenecks in data processing

## Refactoring Requirements

- Convert from Jupyter notebook to modular Python scripts
- Implement proper error handling and logging
- Follow PEP 8 Python coding standards
- Add comprehensive unit and integration tests using pytest
- Optimize data processing for performance
- Create a well-organized project structure with clear separation of concerns.
- Class/OOP approach is optional - focus on functional modularity


## Import Required Libraries


In [3]:
import random
import sqlite3
from datetime import datetime, timedelta

import pandas as pd

In [ ]:
conn = sqlite3.connect("data/retail_sales.db")
print("Database connection created")

## Download sales details.

### branch, articles, sales details fron DB


In [ ]:
# Read branches from database
query = "SELECT * FROM branches"
df_branches = pd.read_sql_query(query, conn)
print("Branches from database:")
print(df_branches)

In [ ]:
# Read articles from database
query = "SELECT * FROM articles"
df_articles = pd.read_sql_query(query, conn)
print("Articles from database:")
print(df_articles)

In [ ]:
# Read sales from database
query = "SELECT * FROM sales"
df_sales = pd.read_sql_query(query, conn)
print("Sales from database:")
print(df_sales)

## Basic Data Exploration


In [ ]:
print("Branches Info:")
print(df_branches.info())
print("\n")
print(df_branches.describe())
print("\n")
print("Missing values:")
print(df_branches.isnull().sum())

In [ ]:
print("Articles Info:")
print(df_articles.info())
print("\n")
print(df_articles.describe())
print("\n")
print("Missing values:")
print(df_articles.isnull().sum())

In [ ]:
print("Sales Info:")
print(df_sales.info())
print("\n")
print(df_sales.describe())
print("\n")
print("Missing values:")
print(df_sales.isnull().sum())

## Simple Data Transformations


In [ ]:
# Merge sales with articles to get price
sales_with_price = df_sales.merge(df_articles, on="article_id", how="left")
print("Sales merged with articles:")
print(sales_with_price.head())

In [ ]:
# Calculate total amount
sales_with_price["total_amount"] = (
    sales_with_price["quantity"] * sales_with_price["price"]
)
print("Total amount calculated:")
print(
    sales_with_price[
        ["transaction_id", "article_name", "quantity", "price", "total_amount"]
    ].head()
)

In [ ]:
# Add month and year columns
sales_with_price["sale_date"] = pd.to_datetime(sales_with_price["sale_date"])
sales_with_price["month"] = sales_with_price["sale_date"].dt.month
sales_with_price["year"] = sales_with_price["sale_date"].dt.year
print("Month and year added:")
print(sales_with_price[["transaction_id", "sale_date", "month", "year"]].head())

## Calculate Business Metrics


In [ ]:
# Total sales per branch
sales_by_branch = sales_with_price.groupby("branch_id")["total_amount"].sum()
print("Total sales per branch:")
print(sales_by_branch)

In [ ]:
# Top selling articles
top_articles = (
    sales_with_price.groupby("article_name")["quantity"]
    .sum()
    .sort_values(ascending=False)
)
print("Top selling articles by quantity:")
print(top_articles)

In [ ]:
# Monthly revenue
monthly_revenue = sales_with_price.groupby(["year", "month"])["total_amount"].sum()
print("Monthly revenue:")
print(monthly_revenue)

In [ ]:
# Revenue by category
category_revenue = sales_with_price.groupby("category")["total_amount"].sum()
print("Revenue by category:")
print(category_revenue)

In [ ]:
# Write sales to database
sales_with_price.to_sql("sales_detail", conn, if_exists="replace", index=False)
print("Sales table created and data inserted")
print("Total records inserted:", len(sales_with_price))

## Perform SQL Queries on Sales Data


In [ ]:
# Query: Total sales by branch
query = """
SELECT b.branch_name, SUM(s.total_amount) as total_sales
FROM sales_detail s
JOIN branches b ON s.branch_id = b.branch_id
GROUP BY b.branch_name
ORDER BY total_sales DESC
"""
result = pd.read_sql_query(query, conn)
print("Total sales by branch:")
print(result)

In [ ]:
# Query: Revenue by article category
query = """
SELECT category, SUM(total_amount) as revenue, COUNT(*) as transaction_count
FROM sales_detail
GROUP BY category
ORDER BY revenue DESC
"""
result = pd.read_sql_query(query, conn)
print("Revenue by category:")
print(result)

In [ ]:
# Query: Top 5 selling articles by revenue
query = """
SELECT article_name, SUM(total_amount) as revenue, SUM(quantity) as total_quantity
FROM sales_detail
GROUP BY article_name
ORDER BY revenue DESC
LIMIT 5
"""
result = pd.read_sql_query(query, conn)
print("Top 5 articles by revenue:")
print(result)

In [ ]:
# Query: Monthly sales trend
query = """
SELECT year, month, SUM(total_amount) as monthly_revenue, COUNT(*) as transaction_count
FROM sales_detail
GROUP BY year, month
ORDER BY year, month
"""
result = pd.read_sql_query(query, conn)
print("Monthly sales trend:")
print(result)

In [ ]:
# Query: Sales performance by city
query = """
SELECT b.city, COUNT(DISTINCT b.branch_id) as num_branches, 
       SUM(s.total_amount) as total_revenue,
       AVG(s.total_amount) as avg_transaction_value
FROM sales_detail s
JOIN branches b ON s.branch_id = b.branch_id
GROUP BY b.city
ORDER BY total_revenue DESC
"""
result = pd.read_sql_query(query, conn)
print("Sales performance by city:")
print(result)

## Write Business Metrics to DB


In [ ]:
sales_by_branch.columns = ["branch_id", "total_sales"]
sales_by_branch.to_sql(
    "metrics_sales_by_branch", conn, if_exists="replace", index=False
)

top_articles.columns = ["article_name", "total_quantity"]
top_articles.to_sql("metrics_top_articles", conn, if_exists="replace", index=False)


monthly_revenue.columns = ["year", "month", "revenue"]
monthly_revenue.to_sql(
    "metrics_monthly_revenue", conn, if_exists="replace", index=False
)

category_revenue.columns = ["category", "revenue"]
category_revenue.to_sql(
    "metrics_category_revenue", conn, if_exists="replace", index=False
)
# Commit changes
conn.commit()
print("\nAll business metrics saved to database successfully!")

## Close Database Connection


In [ ]:
# Close the database connection
conn.close()
print("Database connection closed")